# UE1: Interlab Comparison
A world-wide scientific study requests data from different labs to get as comprehensive analyses as possible for a global study. Per county a different number of labs agree to participate and they agree to submit data of concentration levels of the requested parameter. 

You are the person who must decide if the submitted data can be included to the global study. Consider the first data set to be the data from other countries/continents.

_What do you gave to consider? Can I combine the data to one data set?_ 

_What are you doing with the data?test set: Gaussian distributed? test set(s): variance comparable to other data?_

_What is your result?_